In [22]:
import numpy as np
import pandas as pd

data = pd.read_csv("~/rosetta-antibody-ddgs/raw_datasets/full_data.csv")
del data[data.columns[0]]  # Deleting new index row

# Subsetting for phillips data
data = data[data["Source"].str.contains("Phillips")]
print(data[data["LD"] == 1][data["#PDB"] == "4FQY"])

mut_1 = data[data["LD"] == 1]
mut_2 = data[data["LD"] == 2]
mut_3 = data[data["LD"] == 3]
mut_4 = data[data["LD"] == 4]



       #PDB Mutations  ddG(kcal/mol)                Source  LD
2867   4FQY    H:A59N      -0.433136  Phillips et al. 2021   1
2870   4FQY    H:F95Y      -0.143747  Phillips et al. 2021   1
3037   4FQY    H:N77S       0.653622  Phillips et al. 2021   1
3168   4FQY    H:T58A       1.272136  Phillips et al. 2021   1
3195   4FQY   H:S106Y       0.224343  Phillips et al. 2021   1
3229   4FQY    H:S76T      -0.233936  Phillips et al. 2021   1
3428   4FQY    H:S71T       0.208141  Phillips et al. 2021   1
3533   4FQY    H:T87R       0.137087  Phillips et al. 2021   1
3936   4FQY   H:S106Y       0.476781  Phillips et al. 2021   1
4388   4FQY    H:N30S       0.859490  Phillips et al. 2021   1
4439   4FQY    H:N84S       0.169210  Phillips et al. 2021   1
5333   4FQY    H:N31S       1.394786  Phillips et al. 2021   1
6299   4FQY    H:S57T       2.104520  Phillips et al. 2021   1
7241   4FQY    H:F75S       2.796839  Phillips et al. 2021   1
14161  4FQY    H:I74K       4.674663  Phillips et al. 2

/tmp/ipykernel_4861/4259371715.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(data[data["LD"] == 1][data["#PDB"] == "4FQY"])
